In [1]:
versioninfo()

Julia Version 0.4.0-rc4
Commit 7048c40 (2015-10-07 00:33 UTC)
Platform Info:
  System: Darwin (x86_64-apple-darwin13.4.0)
  CPU: Intel(R) Core(TM) i7-2635QM CPU @ 2.00GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Sandybridge)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.3


In [2]:
type LIFSomas
    Id::Array{Float32,1} # dendritic input current
    Is::Array{Float32,1} # synaptic input current
    u::Array{Float32,1} # membrane potential
    z::Array{Int32,1} # spike count
    θ::Float32 # spiking threshold
    ρ::Float32 # reset potential
    τ::Float32 # membrane time constant
    g_leak::Float32 # leak conductance
end
Base.length(somas::LIFSomas) = length(somas.u)

function LIFSomas(n::Int; θ=0.8, ρ=0.1, τ=20e-3, g_leak=1.0)
    Id = zeros(Float32, n)
    Is = zeros(Float32, n)
    us = zeros(Float32, n)
    zs = zeros(Int32, n)
    LIFSomas(Id, Is, us, zs, θ, ρ, τ, g_leak)
end

LIFSomas

In [3]:
somas = LIFSomas(100)

LIFSomas(Float32[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],Float32[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],Float32[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],Int32[0,0,0,0,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0],0.8f0,0.1f0,0.02f0,1.0f0)

In [6]:
function update(group, dt)
    Id_group = group.Id
    Is_group = group.Is
    u_group = group.u
    z_group = group.z
    θ_group = group.θ
    ρ_group = group.ρ
    τ_group = group.τ
    g_leak_group = group.g_leak
    @simd for i = 1:length(group)
            @inbounds begin
                    I_leak = -(u_group[i]) * g_leak_group * (u_group[i] > 0)
                    dudt = (Id_group[i] + Is_group[i] + I_leak) / τ_group
                    u_group[i] = u_group[i] + dudt * dt
                end
        end
end

update (generic function with 1 method)

In [7]:
update(somas, 1e-3)

In [9]:
@time update(somas, 1e-3)

  0.000005 seconds (4 allocations: 160 bytes)


In [10]:
@code_llvm update(somas, 1e-3)


define void @julia_update_21884(%jl_value_t*, double) {
L:
  %2 = alloca [6 x %jl_value_t*], align 8
  %.sub = getelementptr inbounds [6 x %jl_value_t*]* %2, i64 0, i64 0
  %3 = getelementptr [6 x %jl_value_t*]* %2, i64 0, i64 2
  store %jl_value_t* inttoptr (i64 8 to %jl_value_t*), %jl_value_t** %.sub, align 8
  %4 = load %jl_value_t*** @jl_pgcstack, align 8
  %5 = getelementptr [6 x %jl_value_t*]* %2, i64 0, i64 1
  %.c = bitcast %jl_value_t** %4 to %jl_value_t*
  store %jl_value_t* %.c, %jl_value_t** %5, align 8
  store %jl_value_t** %.sub, %jl_value_t*** @jl_pgcstack, align 8
  store %jl_value_t* null, %jl_value_t** %3, align 8
  %6 = getelementptr [6 x %jl_value_t*]* %2, i64 0, i64 3
  store %jl_value_t* null, %jl_value_t** %6, align 8
  %7 = getelementptr [6 x %jl_value_t*]* %2, i64 0, i64 4
  store %jl_value_t* null, %jl_value_t** %7, align 8
  %8 = getelementptr [6 x %jl_value_t*]* %2, i64 0, i64 5
  store %jl_value_t* null, %jl_value_t** %8, align 8
  %9 = getelementptr inbou

In [11]:
@code_native update(somas, 1e-3)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[6]
Source line: 4
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 4
	subq	$96, %rsp
	movq	$8, -48(%rbp)
	movabsq	$jl_pgcstack, %rax
	movq	(%rax), %rcx
	movq	%rcx, -40(%rbp)
	leaq	-48(%rbp), %rcx
	movq	%rcx, (%rax)
	vxorpd	%xmm1, %xmm1, %xmm1
	vmovupd	%xmm1, -16(%rbp)
	vmovupd	%xmm1, -32(%rbp)
Source line: 2
	movq	(%rdi), %rcx
	movq	%rcx, -32(%rbp)
	xorl	%eax, %eax
Source line: 3
	movq	8(%rdi), %rdx
	movq	%rdx, -24(%rbp)
Source line: 4
	movq	16(%rdi), %rsi
	movq	%rsi, -16(%rbp)
Source line: 9
	vmovss	44(%rdi), %xmm9
Source line: 8
	vmovss	40(%rdi), %xmm4
Source line: 65
	movq	16(%rdi), %rdi
	movq	%rdi, -8(%rbp)
	movq	8(%rdi), %rdi
	testq	%rdi, %rdi
	cmovnsq	%rdi, %rax
Source line: 67
	cmpq	$1, %rax
	leaq	-1(%rax), %rax
	jo	L664
	leaq	1(%rax), %r11
	addq	$1, %rax
	jo	L664
	testq	%r11, %r11
	jle	L638
Source line: 13
	movq	(%rdx), %r8
	movq	(%rcx), %r9
Source line: 12
	movq	(%rsi), %r10
	xorl	%edi, %edi
	movq	%r11, %rdx
	andq	$-8, %r